This notebook is for data management

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import os
import shutil
from IPython.display import Image, clear_output  # to display images

print(f'Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else "CPU"})')

In [ ]:
torch.cuda.is_available()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import subprocess, json

In [ ]:
DIR = os.path.abspath('')
CONTENT_FOLDER = os.path.join(DIR, 'content')
RAW_VIDEO_FOLDER = os.path.join(CONTENT_FOLDER, 'raw_video') # store raw video here
EXTRACTED_VIDEO_FOLDER = os.path.join(CONTENT_FOLDER, 'extracted')
TRIMMED_VIDEO_FOLDER = os.path.join(CONTENT_FOLDER, 'trimmed')

yolo_model = 'yolov5/models/crowdhuman_yolov5m.pt'

for folder in [CONTENT_FOLDER, RAW_VIDEO_FOLDER, EXTRACTED_VIDEO_FOLDER, TRIMMED_VIDEO_FOLDER]:
    try:
        os.makedirs(folder)
    except Exception as e:
        print(e)

In [ ]:
tmp = os.walk(RAW_VIDEO_FOLDER)
raw_video_list = []
class_folder = next(tmp)[1]
for root, dirs, files in tmp:
    raw_video_list.extend(files)
    print(root, dirs, files)

del tmp

In [ ]:
print(class_folder)
print(raw_video_list)

In [ ]:
def get_song_info(file):
    filename, ext = os.path.splitext(file)
    video_folder = file.split('_')[0]
    return filename, video_folder, ext

def time_to_second(s_time):
    '''
    Return the total duration in second from a time string
    Time format: hh:mm:ss 
    '''
    times = s_time.split(':')
    hour = int(times[0])
    minute = int(times[1])
    second = int(times[2])
    second += 3600*hour + 60*minute
    return second

def second_to_time(second):
    '''
    Return time string from second
    Time format: hh:mm:ss 
    '''
    second = int(second)
    hour = int(second / 3600)
    second -= 3600*hour
    
    minute = int(second / 60)
    second -= 60*minute
    
    time = f'{hour:02}:{minute:02}:{second:02}'
    return time

def get_duration_with_ffprobe(filepath):
  '''
  This return a video file's duration (in second) and fps with ffprobe
  '''
  result = subprocess.check_output(f'ffprobe -v quiet -show_streams -select_streams v:0 -of json "{filepath}"',shell=True).decode()
  fields = json.loads(result)['streams'][0]

  duration = float(fields['duration'])
  fps = float(eval(fields['r_frame_rate']))
  return duration, fps

In [ ]:
# reduce video duration
audio_filename, audio_folder, audio_ext = get_song_info(raw_video_list[0])
f_duration, _ = get_duration_with_ffprobe(f'{RAW_VIDEO_FOLDER}/{audio_folder}/{audio_filename}{audio_ext}')
time_interval = 300
s_start = '00:06:30'
i_start = time_to_second(s_start)

In [ ]:

## Make folder
os.makedirs(f'{TRIMMED_VIDEO_FOLDER}/{audio_folder}', exist_ok=True)

while i_start < f_duration:

    i_end = i_start + time_interval
    if i_end > f_duration:
        i_end = f_duration

    s_start = second_to_time(i_start)
    s_end = second_to_time(i_end)

    s_start_no_colon = s_start.replace(':','')
    s_end_no_colon = s_end.replace(':','')
    
    with subprocess.Popen(['ffmpeg', '-y',
                               '-i', f'{RAW_VIDEO_FOLDER}/{audio_folder}/{audio_filename}{audio_ext}',
                               '-ss', s_start,
                               '-to', s_end,
                               '-c', 'copy', f'{TRIMMED_VIDEO_FOLDER}/{audio_folder}/{audio_filename}_s{s_start_no_colon}_e{s_end_no_colon}{audio_ext}',
                               ],shell=False,
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.STDOUT,  # <-- redirect stderr to stdout
                               bufsize=1) as con:
            for line in iter(con.stdout.readline, b''):
                print(line.decode('utf8'),end=''),

    print(s_start, s_end)
    i_start = i_end

In [ ]:
# !python track.py --yolo_model "yolov5/models/crowdhuman_yolov5m.pt" \
# --img 640 --conf-thres 0.5 --source "{TRIMMED_VIDEO_FOLDER}/{audio_folder}/ch08_20210413060707_s000630_e001130.mp4"\
# --skip-frame 25 --save-txt --class 0 --save-crop --project "track/exp/{audio_folder}/{audio_filename}"


In [ ]:
try:
    with subprocess.Popen(['python', 'track.py',
                            '--yolo_model', yolo_model,
                            '--img', '640',
                            '--conf-thres', '0.5',
                            '--max-det', '50',
                            '--source', f'{TRIMMED_VIDEO_FOLDER}/{audio_folder}/ch08_20210413060707_s000630_e001130.mp4',
                            '--skip-frame', '25',
                            '--save-txt',
                            '--class', '0',
                            '--save-crop',
                            '--project', f'{EXTRACTED_VIDEO_FOLDER}/{audio_folder}/{audio_filename}'
                            ],shell=False,
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.STDOUT,  # <-- redirect stderr to stdout
                            bufsize=1) as con:
        for line in iter(con.stdout.readline, b''):
            print(line.decode('utf8'),end='')
except:
    con.terminate()
    raise